# HurdleDMR

HurdleDMR.jl is a Julia implementation of the Hurdle Distributed Multiple Regression (HDMR), as described in:

Kelly, Bryan, Asaf Manela, and Alan Moreira (2018). Text Selection. [Working paper](http://apps.olin.wustl.edu/faculty/manela/kmm/textselection/).

It includes a Julia implementation of the Distributed Multinomial Regression (DMR) model of [Taddy (2015)](https://arxiv.org/abs/1311.6139).

This tutorial explains how to use this package.

## Setup

### Install Julia

First, install Julia itself. The easiest way to do that is from the download site https://julialang.org/downloads/. An alternative is to install JuliaPro from https://juliacomputing.com

Once installed, open julia in a terminal (or in Atom) and add the following packages:

In [ ]:
Pkg.clone("https://github.com/AsafManela/Lasso.jl")
Pkg.clone("https://github.com/AsafManela/HurdleDMR.jl")

Add parallel workers and make package available to workers

In [1]:
addprocs(Sys.CPU_CORES-2)
import HurdleDMR; @everywhere using HurdleDMR

### Example Data

Setup your data into an n-by-p covars matrix, and a (sparse) n-by-d counts matrix. Here we generate some random data.

In [2]:
using CSV, GLM, DataFrames, Distributions
n = 100
p = 3
d = 4

srand(13)
m = 1+rand(Poisson(5),n)
covars = rand(n,p)
ηfn(vi) = exp.([0 + i*sum(vi) for i=1:d])
q = [ηfn(covars[i,:]) for i=1:n]
scale!.(q,ones(n)./sum.(q))
counts = convert(SparseMatrixCSC{Float64,Int},hcat(broadcast((qi,mi)->rand(Multinomial(mi, qi)),q,m)...)')
covarsdf = DataFrame(covars,[:vy, :v1, :v2])

,vy,v1,v2
1,0.693073,0.877116,0.401554
2,0.938163,0.737491,0.997271
3,0.755878,0.743268,0.595892
4,0.191058,0.296443,0.30533
5,0.00753542,0.360474,0.335553
6,0.410974,0.773871,0.657641
7,0.279942,0.154284,0.321258
8,0.208454,0.849653,0.22147
9,0.639872,0.926706,0.444675
10,0.269132,0.83785,0.0137366


## Distributed Multinomial Regression (DMR)

The Distributed Multinomial Regression (DMR) model of Taddy (2015) is a highly scalable
approximation to the Multinomial using distributed (independent, parallel)
Poisson regressions, one for each of the d categories (columns) of a large `counts` matrix,
on the `covars`.

To fit a DMR:

In [3]:
m = dmr(covars, counts)

INFO: fitting 100 observations on 4 categories, 3 covariates 
INFO: distributed poisson run on local cluster with 18 nodes


HurdleDMR.DMRCoefs([-0.277378 -1.10612 -1.18168 -0.746987; -3.94502 -1.7695 -0.387265 0.355981; -2.83456 -1.82878 0.0 0.212385; -2.72819 -0.983448 -0.346102 0.247442], true, 100, 4, 3)

or with a dataframe and formula

In [4]:
mf = @model(c ~ vy + v1 + v2)
m = fit(DMR, mf, covarsdf, counts)

INFO: fitting 100 observations on 4 categories, 3 covariates 
INFO: distributed poisson run on local cluster with 18 nodes


HurdleDMR.DMRCoefs([-0.277378 -1.10612 -1.18168 -0.746987; -3.94502 -1.7695 -0.387265 0.355981; -2.83456 -1.82878 0.0 0.212385; -2.72819 -0.983448 -0.346102 0.247442], true, 100, 4, 3)

in either case we can get the coefficients matrix for each variable + intercept as usual with

In [5]:
coef(m)

4×4 SharedArray{Float64,2}:
 -0.277378  -1.10612   -1.18168   -0.746987
 -3.94502   -1.7695    -0.387265   0.355981
 -2.83456   -1.82878    0.0        0.212385
 -2.72819   -0.983448  -0.346102   0.247442

By default we only return the AICc maximizing coefficients.
To also get back the entire regulatrization paths, run

In [6]:
paths = fit(DMRPaths, mf, covarsdf, counts)

INFO: fitting 100 observations on 4 categories, 3 covariates 
INFO: distributed poisson run on remote cluster with 18 nodes


HurdleDMR.DMRPaths(Nullable{Lasso.GammaLassoPath}[Poisson GammaLassoPath (56) solutions for 4 predictors in 330 iterations):
                λ   pct_dev ncoefs
 [1]     0.108369       0.0      0
 [2]     0.098742 0.0275025      1
 [3]      0.08997 0.0509363      1
 [4]    0.0819773 0.0793996      2
 [5]    0.0746947  0.112825      3
 [6]     0.068059  0.148402      3
 [7]    0.0620129  0.179008      3
 [8]    0.0565038    0.2054      3
 [9]    0.0514842  0.228196      3
[10]    0.0469105  0.247908      3
[11]    0.0427431  0.264961      3
[12]    0.0389459  0.279718      3
[13]     0.035486  0.292484      3
[14]    0.0323336  0.303522      3
[15]    0.0294611   0.31306      3
[16]    0.0268439  0.321293      3
[17]    0.0244591  0.328392      3
[18]    0.0222863  0.334505      3
[19]    0.0203064  0.339762      3
[20]    0.0185024  0.344276      3
[21]    0.0168587  0.348147      3
[22]    0.0153611  0.351461      3
[23]    0.0139964  0.354294      3
[24]     0.012753  0.356711      3


we can now select, for example the coefficients that minimize CV mse (takes a while)

In [7]:
coef(paths; select=:CVmin)

4×4 Array{Float64,2}:
 -1.04233  -1.27898   -1.1979    -0.733303
 -3.08667  -1.61234   -0.368288   0.346255
 -2.03017  -1.65467    0.0        0.202824
 -1.92495  -0.857511  -0.329399   0.240294

## Hurdle Distributed Multiple Regression (HDMR)

For highly sparse counts, as is often the case with text that is selected for
various reasons, the Hurdle Distributed Multiple Regression (HDMR) model of
Kelly, Manela, and Moreira (2018), may be superior to the DMR. It approximates
a higher dispersion Multinomial using distributed (independent, parallel)
Hurdle regressions, one for each of the d categories (columns) of a large `counts` matrix,
on the `covars`. It allows a potentially different sets of covariates to explain
category inclusion ($h=1{c>0}$), and repetition ($c>0$).

Both the model for zeroes and for positive counts are regularized by default,
using `GammaLassoPath`, picking the AICc optimal segment of the regularization
path.

HDMR can be fitted:

In [8]:
m = hdmr(covars, counts; inpos=1:2, inzero=1:3)

INFO: fitting 100 observations on 4 categories 
2 covariates for positive and 3 for zero counts
INFO: distributed hurdle run on local cluster with 18 nodes


HurdleDMR.HDMRCoefs([-2.97869 -3.39104 -1.37637 -0.690629; 0.0 0.0 -0.724477 0.426105; 0.0 0.0 0.0 0.244869], [0.629668 -0.00316568 -0.521496 2.98134; -4.68583 -2.4084 0.0 0.0; -3.37301 -2.30744 0.0 0.0; -3.37706 -1.50431 0.0 0.0], true, 100, 4, 1:2, 1:3)

or with a dataframe and formula

In [9]:
mf = @model(h ~ vy + v1 + v2, c ~ vy + v1)
m = fit(HDMR, mf, covarsdf, counts)

INFO: fitting 100 observations on 4 categories 
2 covariates for positive and 3 for zero counts
INFO: distributed hurdle run on local cluster with 18 nodes


HurdleDMR.HDMRCoefs([-2.97869 -3.39104 -1.37637 -0.690629; 0.0 0.0 -0.724477 0.426105; 0.0 0.0 0.0 0.244869], [0.629668 -0.00316568 -0.521496 2.98134; -4.68583 -2.4084 0.0 0.0; -3.37301 -2.30744 0.0 0.0; -3.37706 -1.50431 0.0 0.0], true, 100, 4, [1, 2], [1, 2, 3])

where the h ~ equation is the model for zeros (hurdle crossing) and c ~ is the model for positive counts

in either case we can get the coefficients matrix for each variable + intercept as usual with

In [10]:
coefspos, coefszero = coef(m)

([-2.97869 -3.39104 -1.37637 -0.690629; 0.0 0.0 -0.724477 0.426105; 0.0 0.0 0.0 0.244869], [0.629668 -0.00316568 -0.521496 2.98134; -4.68583 -2.4084 0.0 0.0; -3.37301 -2.30744 0.0 0.0; -3.37706 -1.50431 0.0 0.0])

By default we only return the AICc maximizing coefficients.
To also get back the entire regulatrization paths, run

In [11]:
paths = fit(HDMRPaths, mf, covarsdf, counts)

coef(paths; select=:all)

INFO: fitting 100 observations on 4 categories 
2 covariates for positive and 3 for zero counts
INFO: distributed hurdle run on remote cluster with 18 nodes


([-2.97869 0.0 0.0; -2.93456 -0.15307 0.0; … ; 0.0 0.0 0.0; 0.0 0.0 0.0]

[-3.39104 0.0 0.0; -3.29061 0.0 -0.251553; … ; -1.04786 -2.52399 -5.36357; -1.04459 -2.52915 -5.37187]

[-1.68781 0.0 0.0; -1.65155 -0.0792342 0.0; … ; 0.0 0.0 0.0; 0.0 0.0 0.0]

[-0.352618 0.0 0.0; -0.370878 0.037995 0.0; … ; 0.0 0.0 0.0; 0.0 0.0 0.0], [-3.87231 0.0 0.0 0.0; -3.72971 -0.30193 0.0 0.0; … ; 0.62351 -4.679 -3.36716 -3.37106; 0.629668 -4.68583 -3.37301 -3.37706]

[-2.86015 0.0 0.0 0.0; -2.74269 -0.19503 -0.0481219 0.0; … ; 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0]

[-0.521496 0.0 0.0 0.0; -0.448339 0.0 0.0 -0.13552; … ; 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0]

[2.98134 0.0 0.0 0.0; 2.86604 0.0 0.248068 0.0; … ; 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0])

## Sufficient reduction projection

A sufficient reduction projection summarizes the counts, much like a sufficient
statistic, and is useful for reducing the d dimensional counts in a potentially
much lower dimension matrix `z`.

To get a sufficient reduction projection in direction of vy for the above
example

In [12]:
z = srproj(m,counts,1,1)

100×3 Array{Float64,2}:
  0.311047    0.0       10.0
  0.311047    0.0       10.0
  0.426105    0.0        3.0
 -0.260452   -0.802801   9.0
  0.163324   -1.56194    6.0
  0.261736    0.0        7.0
  0.304361   -1.2042     7.0
 -0.0670014   0.0        7.0
  0.426105    0.0        8.0
  0.17042     0.0        9.0
  0.298263    0.0        9.0
  0.426105    0.0        4.0
  0.13846     0.0        4.0
  ⋮                         
 -0.119349   -1.56194    5.0
  0.426105    0.0        4.0
 -0.149186    0.0        4.0
  0.0923065  -0.802801   6.0
  0.110768   -0.802801   5.0
  0.13846     0.0        4.0
  0.11231     0.0       11.0
  0.0364953  -0.802801   7.0
 -0.0586262  -0.802801   8.0
  0.426105    0.0        2.0
  0.0923065  -1.77356    6.0
  0.261736    0.0        7.0

Here, the first column is the SR projection from the model for positive counts, the second is the the SR projection from the model for hurdle crossing (zeros), and the third is the total count for each observation.

## Counts Inverse Regression (CIR)

Counts inverse regression allows us to predict a covariate with the counts and other covariates.
Here we use hdmr for the backward regression and another model for the forward regression.
This can be accomplished with a single command, by fitting a CIR{HDMR,FM} where the forward model is FM <: RegressionModel.

In [13]:
cir = fit(CIR{HDMR,LinearModel},mf,covarsdf,counts,:vy; nocounts=true)

INFO: fitting 100 observations on 4 categories 
2 covariates for positive and 3 for zero counts
INFO: distributed hurdle run on local cluster with 18 nodes


HurdleDMR.CIR{HurdleDMR.HDMR,GLM.LinearModel}(1, [1, 2], HurdleDMR.HDMRCoefs([-2.97869 -3.39104 -1.37637 -0.690629; 0.0 0.0 -0.724477 0.426105; 0.0 0.0 0.0 0.244869], [0.629668 -0.00316568 -0.521496 2.98134; -4.68583 -2.4084 0.0 0.0; -3.37301 -2.30744 0.0 0.0; -3.37706 -1.50431 0.0 0.0], true, 100, 4, [1, 2], [1, 2, 3]), GLM.LinearModel{GLM.LmResp{Array{Float64,1}},GLM.DensePredChol{Float64,Base.LinAlg.Cholesky{Float64,Array{Float64,2}}}}:

Coefficients:
       Estimate Std.Error   t value Pr(>|t|)
x1     0.596995  0.108965   5.47876    <1e-6
x2    -0.165407 0.0953801  -1.73418   0.0862
x3    -0.059985 0.0933614 -0.642503   0.5221
x4     0.283205  0.126589   2.23721   0.0276
x5     0.160959 0.0471665   3.41257   0.0010
x6   0.00293183 0.0116717  0.251192   0.8022

, GLM.LinearModel{GLM.LmResp{Array{Float64,1}},GLM.DensePredChol{Float64,Base.LinAlg.Cholesky{Float64,Array{Float64,2}}}}:

Coefficients:
       Estimate Std.Error   t value Pr(>|t|)
x1     0.456718 0.0713401   6.40198    <1e

where the ```nocounts=true``` means we also fit a benchmark model without counts.

we can get the forward and backward model coefficients with

In [14]:
coefbwd(cir)

([-2.97869 -3.39104 -1.37637 -0.690629; 0.0 0.0 -0.724477 0.426105; 0.0 0.0 0.0 0.244869], [0.629668 -0.00316568 -0.521496 2.98134; -4.68583 -2.4084 0.0 0.0; -3.37301 -2.30744 0.0 0.0; -3.37706 -1.50431 0.0 0.0])

In [15]:
coeffwd(cir)

6-element Array{Float64,1}:
  0.596995  
 -0.165407  
 -0.059985  
  0.283205  
  0.160959  
  0.00293183

The fitted model can be used to predict vy with new data

In [16]:
yhat = predict(cir, covarsdf[1:10,:], counts[1:10,:])

10-element Array{Union{Float64, Missings.Missing},1}:
 0.545235
 0.532596
 0.567779
 0.353053
 0.329678
 0.524191
 0.465097
 0.444719
 0.561167
 0.532235

We can also predict only with the other covariates, which in this case
is just a linear regression

In [17]:
yhat_nocounts = predict(cir, covarsdf[1:10,:], counts[1:10,:]; nocounts=true)

10-element Array{Union{Float64, Missings.Missing},1}:
 0.457317
 0.517999
 0.480777
 0.472548
 0.472676
 0.485196
 0.479915
 0.441864
 0.459232
 0.423214

Kelly, Manela, and Moreira (2018) show that the differences between DMR and HDMR can be substantial in some cases, especially when the counts data is highly sparse.

Please reference the paper for additional details and example applications.